# Reionised bubbles around primordial galaxies

## Introduction

This is the second ipython notebook to be followed alongside the exercise sheet. Again, the following code only serves as a starting point, feel free to adjust it as you see fit. Software requirements are discussed in the first notebook.

In [ ]:
# Load necessary packages
import os

import numpy as np
import math

import lmfit
from spectres import spectres

import matplotlib
%matplotlib inline
print("Matplotlib", matplotlib.__version__, "(backend: " + matplotlib.get_backend() + ')')
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("ticks")

# Load matplotlib style file
plt.style.use("../auxiliary/bubble.mplstyle")
def_linewidth = plt.rcParams["lines.linewidth"]
def_markersize = plt.rcParams["lines.markersize"]

pformat = ".png"
dpi = 300

# Import astropy cosmology, given H0 and Omega_matter (Planck Collaboration 2020)
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315, Ob0=0.02237/0.674**2, Tcmb0=2.726)

from astropy.io import fits
from astroquery.mast import Observations

## Gather spectroscopic data and measure the redshift and line fluxes

First, we will define a function to download the data from MAST using `astroquery`.

In [ ]:
def download_data(obs_id, force_download=False):
    if force_download or not os.path.isfile("../data/mastDownload/HLSP/{obs_id}/{obs_id}_x1d.fits".format(obs_id=obs_id)):
        obs_table = Observations.query_criteria(provenance_name="JADES", instrument_name="NIRSPEC", obs_id=obs_id)
        data_products = Observations.get_product_list(obs_table)
        Observations.download_products(data_products, download_dir="../data/")
    else:
        print("Data products already downloaded.")

filter_gratings = ["clear-prism", "f070lp-g140m", "f290lp-g395m"]

Next, we define a function to plot the spectra.

In [ ]:
# Lyα wavelength
wl_Lya = np.nan # Angstrom
# Convenient scaling factor for flux densities
conv = 1e20

colors1 = sns.color_palette("Set1", 9)

def plot_spectra(ID, wl_obs_dict, flux_dict, flux_err_dict, redshift=None, savefig=False):
    fig = plt.figure(figsize=(8.27, 8.27/2))
    gs = fig.add_gridspec(nrows=2, ncols=2, hspace=0.2, wspace=0.2, width_ratios=[1, 2])
    axes = {}

    # Create an (invisible) label axes spanning all subplots
    axes["label"] = fig.add_subplot(gs[:, :])
    ax_r = axes["label"].secondary_xaxis("top")
    for ax in [axes["label"], ax_r]:
        ax.tick_params(axis="both", which="both", bottom=False, top=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        for spine in ax.spines.values():
            spine.set_visible(False)

    axes["label"].set_xlabel(r"Observed wavelength $\lambda_\mathrm{obs} \, (\mathrm{\mu m})$", labelpad=20)
    axes["label"].set_ylabel(r"Flux density $F_\lambda \, (10^{{{:d}}} \, \mathrm{{erg \, s^{{-1}} \, cm^{{-2}} \, \AA^{{-1}}}})$".format(int(-np.log10(conv))), labelpad=25)

    axes["label"].annotate(text="ID {}".format(ID), xy=(0.5, 1), xytext=(0, 8),
                            xycoords="axes fraction", textcoords="offset points", va="top", ha="center")

    # Create different axes for each disperser
    for fg, c in zip(filter_gratings, colors1):
        rowi = 0 if fg == "clear-prism" else 1
        coli = slice(None) if fg == "clear-prism" else list(filter_gratings).index(fg)-1
        
        axes[fg] = fig.add_subplot(gs[rowi, coli])

        axes[fg].axhline(y=0, color='k', alpha=0.8)
        axes[fg].annotate(text=fg, xy=(1, 1), xytext=(-8, -8), xycoords="axes fraction", textcoords="offset points",
                            va="top", ha="right", color=c)

        wl_obs = wl_obs_dict[fg]
        flux = flux_dict[fg]
        flux_err = flux_err_dict[fg]
        
        if redshift is not None:
            obs_wl_Lya = wl_Lya / 1e4 * (1.0+redshift)
            if obs_wl_Lya > np.min(wl_obs) and obs_wl_Lya < np.max(wl_obs):
                axes[fg].axvline(obs_wl_Lya, 0, 0.9, linestyle='--', color='k', alpha=0.8)
                axes[fg].annotate(text="Lyα", xy=(obs_wl_Lya, 1), xytext=(0, -4),
                                xycoords=axes[fg].get_xaxis_transform(), textcoords="offset points",
                                va="top", ha="center", size="small")
        
        axes[fg].plot(wl_obs, flux*conv, drawstyle="steps-mid", color=c, alpha=0.8, zorder=1)
        axes[fg].fill_between(wl_obs, (flux-flux_err)*conv, (flux+flux_err)*conv, step="mid",
                            edgecolor="None", facecolor=c, alpha=0.2, zorder=0.5)

        if redshift is not None and fg == "f070lp-g140m":
            axes[fg].set_xlim(0.98*obs_wl_Lya, 1.02*obs_wl_Lya)
        else:
            axes[fg].set_xlim(np.min(wl_obs), np.max(wl_obs))
        axes[fg].set_ylim(1.25*np.nanmin(flux[wl_obs > 1.0]*conv), 1.25*np.nanmax(flux[wl_obs > 1.0]*conv))

    if savefig:
        fig.savefig("../plots/spectra_{}".format(ID) + pformat, dpi=dpi, bbox_inches="tight")
    
    return (fig, axes)

## Measure redshift and line fluxes for ID 1899 in GOODS-N

First, load in the data and plot the spectra.

In [ ]:
field = "goods-n"
tier = "mediumhst"
ID = 1899

wl_obs_dict = {}
flux_dict = {}
flux_err_dict = {}

for fg in filter_gratings:
    print("Downloading {} spectrum of ID {} in {}...".format(fg, ID, field))
    obs_id = "hlsp_jades_jwst_nirspec_{}-{}-{:08d}_{}_v1.0".format(field, tier, ID, fg)
    download_data(obs_id)
    print("Done!")

    print("Reading in {} spectrum of ID {} in {}...".format(fg, ID, field), end=' ')
    with fits.open("../data/mastDownload/HLSP/{obs_id}/{obs_id}_x1d.fits".format(obs_id=obs_id)) as hdulist_1D:
        wl_obs_dict[fg] = hdulist_1D["EXTRACT1D"].data["WAVELENGTH"] # in μm
        flux_dict[fg] = hdulist_1D["EXTRACT1D"].data["FLUX"] # in erg/s/cm^2/Å
        flux_err_dict[fg] = hdulist_1D["EXTRACT1D"].data["FLUX_ERR"] # in erg/s/cm^2/Å
    print("Done!")

fig = plot_spectra(ID, wl_obs_dict, flux_dict, flux_err_dict, savefig=True)[0]
plt.show()
plt.close(fig)

Next, define a model of Gaussian emission line profiles.

In [ ]:
# FWHM of the line spread function, Δλ = λ/R, is ~2.35 times the dispersion σ
FWHM_to_sigma = (2.0 * np.sqrt(2.0 * np.log(2.0)))

def Gaussian(A, x0, y0, sigma, x):
    # Gaussian profile with amplitude A, mean x0, variance sigma^2, and offset y0
    return A / (np.sqrt(2*np.pi)*sigma) * np.exp( -(x-x0)**2 / (2.0*sigma**2) ) + y0

def emission_line_model(params, x, data=None, data_err=None, line_dict={}, spectral_resolution=10000.0):
    if isinstance(params, type(lmfit.Parameters())):
        params = params.valuesdict()

    # Start by calculating the required resolution of the model wavelength grid,
    # which we want to properly sample the line spread function, requiring a
    # step size of at least 3x the dispersion (σ)
    dx_model = np.min(x) / spectral_resolution / FWHM_to_sigma / 3.0
    # Initialise high-resolution model wavelength grid
    x_model = np.arange(np.min(x)-10*dx_model, np.max(x)+10*dx_model, dx_model)
    # Initialise a model flux-density array at constant continuum level
    y_model = np.tile(float(params["cont"]), x_model.shape)

    for line, wl in line_dict.items():
        # Observed (redshifted) wavelength of the line
        wl_obs = do_something_with(wl, params['z'])
        if "deltav_{}".format(line) in params:
            # If a velocity offset is specified, shift the line centre
            wl_obs /= do_something_else_with(params["deltav_{}".format(line)])
        
        # Convert intrinsic velocity width to wavelength space
        sigma_l = do_something_else_with(wl_obs, params["sigma_v"])
        # Convolve with a Gaussian line spread function (dispersion σ adds quadratically)
        sigma_conv = do_something_else_with(sigma_l, wl_obs / spectral_resolution / FWHM_to_sigma)
        # Add the line to the model spectrum
        y_model += Gaussian(A=params["amplitude_{}".format(line)], x0=wl_obs, y0=0, sigma=sigma_conv, x=x_model)

    if data is None:
        # Return the high-resolution model directly
        return (x_model, y_model)
    else:
        # Rebin the high-resolution model to the observed data wavelength bins
        model = spectres(x, x_model, y_model)
        if data_err is None:
            return model - data
        else:
            return (model-data) / data_err

Set up the model to measure the redshift of ID 1899 with the rest-frame optical lines.

In [ ]:
# Verify the currently loaded data is still for ID 1899
assert ID == 1899

# Set up parameters, starting with redshift (z), continuum level (cont), and line width in velocity space (sigma_v)
params = lmfit.Parameters()

z_guess = np.nan
z_min = np.nan
z_max = np.nan

#                NAME           VALUE           VARY        MIN     MAX         EXPR        BRUTE_STEP
params.add_many(('z',           z_guess,        True,       z_min,  z_max,      None,       None),
                ("cont",        0.0,            False,      None,   None,       None,       None),
                ("sigma_v",     100.0,          True,       0,      500,        None,       None))

# Conversion from μm to Å
lconv = 1e4

# Select finite subset of the G395M data to fit to
wl_select = (wl_obs_dict["f290lp-g395m"] > np.nan) * (wl_obs_dict["f290lp-g395m"] < np.nan)
x_fit = wl_obs_dict["f290lp-g395m"][wl_select] * lconv # from μm to Å
y_fit = flux_dict["f290lp-g395m"][wl_select] * conv # in 1e-21 erg/s/cm^2/Å
y_fit_err = selected_flux_errs_in_same_format_as_flux # in 1e-21 erg/s/cm^2/Å

# Assume a constant spectral resolution of R = ...
spectral_resolution = np.nan

# Dictionary containing rest-frame wavelengths of lines to be fit
line_dict = {"Hbeta": np.nan, "OIII4960": np.nan, "OIII5008": np.nan}

# For each line, add amplitude parameter
for line in line_dict:
    expr = "amplitude_OIII4960 * 2.98" if line == "OIII5008" else None
    params.add("amplitude_{}".format(line), value=reasonable_initial_amplitude, vary=True, min=0, max=1e4, expr=expr)

# Plot an example model
fig, axes = plot_spectra(ID, wl_obs_dict, flux_dict, flux_err_dict, redshift=z_guess)

text = "Redshift guess: {:.3f}".format(z_guess)
text += "\nExpected precision: {:.5f}".format(something_to_do_with(line_dict["OIII4960"], z_guess, spectral_resolution, FWHM_to_sigma))
print(text)
axes["f290lp-g395m"].annotate(text=text, xy=(0, 1), xytext=(8, -8),
                              xycoords="axes fraction", textcoords="offset points", va="top", ha="left")
# Using the initial (guessed) parameter values, get the high-resolution model
x_model, y_model = emission_line_model(params.copy(), **some_other_parameters)
axes["f290lp-g395m"].plot(x_model/lconv, y_model, color='k', alpha=0.8)

plt.show()
plt.close(fig)

Next, fit the model.

In [ ]:
# Verify the currently loaded data is still for ID 1899
assert ID == 1899

# Fit the model to the data
fit_result = lmfit.minimize(emission_line_model, params, method="leastsq",
                            args=(x_fit,), kws=dict(data=y_fit, data_err=y_fit_err,
                            line_dict=line_dict, spectral_resolution=spectral_resolution))
print(fit_result.message)
fit_result.params.pretty_print()
print(lmfit.fit_report(fit_result))

Having performed the fitting procedure, plot the results.

In [ ]:
# Verify the currently loaded data is still for ID 1899
assert ID == 1899

# Read in the best-fit redshift and its uncertainty
fit_values = fit_result.params.valuesdict()
z_fit = fit_values['z']
z_fit_err = fit_result.params.get('z').stderr
F_Hbeta_fit = resulting_Hbeta_amplitude
F_Hbeta_fit_err = resulting_Hbeta_amplitude_err

# Show the best-fit high-resolution model curve
fig, axes = plot_spectra(ID, wl_obs_dict, flux_dict, flux_err_dict, redshift=z_fit)

text = "Measured redshift: {:.{prec}f} ± {:.{prec}f}".format(z_fit, z_fit_err, prec=max(0, 2-math.floor(np.log10(z_fit_err))))
print(text)
axes["f290lp-g395m"].annotate(text=text, xy=(0, 1), xytext=(8, -8), xycoords="axes fraction", textcoords="offset points",
                                va="top", ha="left", size="small")

x_model, y_model = emission_line_model(fit_values, x=x_fit, line_dict=line_dict, spectral_resolution=spectral_resolution)
axes["f290lp-g395m"].plot(x_model/lconv, y_model, color='k', alpha=0.8)

axes["f290lp-g395m"].set_xlim(0.95*line_dict["OIII4960"]*(1.0+z_fit)/lconv, 1.05*line_dict["OIII5008"]*(1.0+z_fit)/lconv)

plt.show()
plt.close(fig)

With a measured redshift, measure the flux of the Lyα line.

In [ ]:
# Verify the currently loaded data is still for ID 1899
assert ID == 1899

# Set up parameters, starting with redshift (z), continuum level (cont), and line width in velocity space (sigma_v)
params = lmfit.Parameters()

#                NAME           VALUE           VARY        MIN     MAX         EXPR        BRUTE_STEP
params.add_many(('z',           z_fit,          False,      None,   None,       None,       None),
                ("cont",        0.0,            False,      None,   None,       None,       None),
                ("sigma_v",     100.0,          True,       0,      500,        None,       None))

# Given the lower SNR in the G395M spectrum, this time we will fit to the
# lower-resolution PRISM data – again select finite subset to fit to
wl_select = (wl_obs_dict["f070lp-g140m"] > np.nan) * (wl_obs_dict["f070lp-g140m"] < np.nan)
x_fit = wl_obs_dict["f070lp-g140m"][wl_select] * lconv # from μm to Å
y_fit = flux_dict["f070lp-g140m"][wl_select] * conv # in 1e-21 erg/s/cm^2/Å
y_fit_err = flux_err_dict["f070lp-g140m"][wl_select] * conv # in 1e-21 erg/s/cm^2/Å

# Assume a constant spectral resolution of R = ...
spectral_resolution = np.nan

# Dictionary containing rest-frame wavelengths of lines to be fit
line_dict = {"Lya": wl_Lya}

# Add amplitude parameter and velocity offset
for line in line_dict:
    params.add("amplitude_{}".format(line), value=reasonable_initial_amplitude, vary=True, min=0, max=1e4)
    params.add("deltav_{}".format(line), value=reasonable_initial_velocity_offset, vary=True, min=0, max=300)

# Plot an example model
fig, axes = plot_spectra(ID, wl_obs_dict, flux_dict, flux_err_dict, redshift=z_fit)

# Using the initial (guessed) parameter values, get the high-resolution model
x_model, y_model = emission_line_model(params.copy(), **some_other_parameters)
axes["f070lp-g140m"].plot(x_model/lconv, y_model, color='k', alpha=0.8)

plt.show()
plt.close(fig)

Next, fit the model.

In [ ]:
# Verify the currently loaded data is still for ID 10013682
assert ID == 1899

# Fit the model to the data
fit_result = lmfit.minimize(emission_line_model, params, method="leastsq",
                            args=(x_fit,), kws=dict(data=y_fit, data_err=y_fit_err,
                            line_dict=line_dict, spectral_resolution=spectral_resolution))
print(fit_result.message)
fit_result.params.pretty_print()
print(lmfit.fit_report(fit_result))

Having performed the fitting procedure, plot the results.

In [ ]:
# Verify the currently loaded data is still for ID 1899
assert ID == 1899

# Read in the best-fit redshift and its uncertainty
fit_values = fit_result.params.valuesdict()
F_Lya_fit = fit_values["amplitude_Lya"]
F_Lya_fit_err = fit_result.params.get("amplitude_Lya").stderr
deltav_Lya_fit = resulting_velocity_offset
deltav_Lya_fit_err = resulting_velocity_offset_err

Lya_Hbeta_ratio = F_Lya_fit / F_Hbeta_fit
Lya_Hbeta_ratio_err = error_propagation_of(F_Lya_fit_err, F_Hbeta_fit_err)
f_esc_Lya = convert(Lya_Hbeta_ratio)
f_esc_Lya_err = convert(Lya_Hbeta_ratio_err)

# Show the best-fit high-resolution model curve
fig, axes = plot_spectra(ID, wl_obs_dict, flux_dict, flux_err_dict, redshift=z_fit)

text = "Lyα measurements"
text += "\nFlux: {:.{prec}f} ± {:.{prec}f} x 10^-20 erg/s/cm^2".format(F_Lya_fit, F_Lya_fit_err, prec=max(0, 2-math.floor(np.log10(F_Lya_fit_err))))
text += "\nΔv: {:.{prec}f} ± {:.{prec}f} km/s".format(deltav_Lya_fit, deltav_Lya_fit_err, prec=max(0, 2-math.floor(np.log10(deltav_Lya_fit_err))))
text += "\nFlux ratio with Hβ: {:.2f} ± {:.2f}".format(Lya_Hbeta_ratio, Lya_Hbeta_ratio_err)
text += "\nEscape fraction: {:.2f} ± {:.2f}".format(f_esc_Lya, f_esc_Lya_err)
print(text)
axes["clear-prism"].annotate(text=text, xy=(0.5, 1), xytext=(0, -8), xycoords="axes fraction", textcoords="offset points",
                                va="top", ha="center", size="small")

x_model, y_model = emission_line_model(fit_values, x=x_fit, line_dict=line_dict, spectral_resolution=spectral_resolution)
axes["f070lp-g140m"].plot(x_model/lconv, y_model, color='k', alpha=0.8)

plt.show()
plt.close(fig)

## Measure redshift and line fluxes for ID 10013682 in GOODS-S

First, load in the data and plot the spectra.

In [ ]:
# Repeat the exercise for ID 10013682 in GOODS-S
field = "goods-s"
tier = "deephst"
ID = 10013682

wl_obs_dict = {}
flux_dict = {}
flux_err_dict = {}

for fg in filter_gratings:
    print("Downloading {} spectrum of ID {} in {}...".format(fg, ID, field))
    obs_id = "hlsp_jades_jwst_nirspec_{}-{}-{:08d}_{}_v1.0".format(field, tier, ID, fg)
    download_data(obs_id)
    print("Done!")

    print("Reading in {} spectrum of ID {} in {}...".format(fg, ID, field), end=' ')
    with fits.open("../data/mastDownload/HLSP/{obs_id}/{obs_id}_x1d.fits".format(obs_id=obs_id)) as hdulist_1D:
        wl_obs_dict[fg] = hdulist_1D["EXTRACT1D"].data["WAVELENGTH"] # in μm
        flux_dict[fg] = hdulist_1D["EXTRACT1D"].data["FLUX"] # in erg/s/cm^2/Å
        flux_err_dict[fg] = hdulist_1D["EXTRACT1D"].data["FLUX_ERR"] # in erg/s/cm^2/Å
    print("Done!")

fig = plot_spectra(ID, wl_obs_dict, flux_dict, flux_err_dict, savefig=True)[0]
plt.show()
plt.close(fig)

Set up the model to measure the redshift of ID 1899 with the rest-frame optical lines.

In [ ]:
# Verify the currently loaded data is still for ID 10013682
assert ID == 10013682

# Set up parameters, starting with redshift (z), continuum level (cont), and line width in velocity space (sigma_v)
params = lmfit.Parameters()

z_guess = np.nan
z_min = np.nan
z_max = np.nan

#                NAME           VALUE           VARY        MIN     MAX         EXPR        BRUTE_STEP
params.add_many(('z',           z_guess,        True,       z_min,  z_max,      None,       None),
                ("cont",        0.0,            True,       0,      10,         None,       None),
                ("sigma_v",     100.0,          True,       0,      500,        None,       None))

# Given the lower SNR in the G395M spectrum, this time we will fit to the
# lower-resolution PRISM data – again select finite subset to fit to
wl_select = (wl_obs_dict["clear-prism"] > np.nan) * (wl_obs_dict["clear-prism"] < 4.3)
selected_x_fit_as_before # in 1e-21 erg/s/cm^2/Å
selected_y_fit_as_before # in 1e-21 erg/s/cm^2/Å
selected_y_fit_err_as_before # in 1e-21 erg/s/cm^2/Å

# Assume a constant spectral resolution of R = ...
spectral_resolution = np.nan

# Dictionary containing rest-frame wavelengths of lines to be fit
line_dict = dict_as_before

# For each line, add amplitude parameter
for line in line_dict:
    expr = "amplitude_OIII4960 * 2.98" if line == "OIII5008" else None
    params.add("amplitude_{}".format(line), value=reasonable_initial_amplitude, vary=True, min=0, max=1e4, expr=expr)

# Plot an example model
plot_spectra_and_model_as_before()

plt.show()
plt.close(fig)

Next, fit the model.

In [ ]:
# Verify the currently loaded data is still for ID 10013682
assert ID == 10013682

# Fit the model to the data
fit_result = lmfit.minimize(emission_line_model, params, method="leastsq",
                            args=(x_fit,), kws=dict(data=y_fit, data_err=y_fit_err,
                            line_dict=line_dict, spectral_resolution=spectral_resolution))
print(fit_result.message)
fit_result.params.pretty_print()
print(lmfit.fit_report(fit_result))

Having performed the fitting procedure, plot the results.

In [ ]:
# Verify the currently loaded data is still for ID 10013682
assert ID == 10013682

# Read in the best-fit redshift and its uncertainty
work_out_fit_values_as_before()

# Show the best-fit high-resolution model curve
plot_spectra_and_best_fit_model_as_before()

plt.show()
plt.close(fig)

With a measured redshift, measure the flux of the Lyα line.

In [ ]:
# Verify the currently loaded data is still for ID 10013682
assert ID == 10013682

# Set up parameters, starting with redshift (z), continuum level (cont), and line width in velocity space (sigma_v)
set_up_lmfit_as_before()

# Plot an example model
plot_spectra_and_model_as_before()

plt.show()
plt.close(fig)

Next, fit the model.

In [ ]:
# Verify the currently loaded data is still for ID 10013682
assert ID == 10013682

# Fit the model to the data
fit_result = lmfit.minimize(emission_line_model, params, method="leastsq",
                            args=(x_fit,), kws=dict(data=y_fit, data_err=y_fit_err,
                            line_dict=line_dict, spectral_resolution=spectral_resolution))
print(fit_result.message)
fit_result.params.pretty_print()
print(lmfit.fit_report(fit_result))

Having performed the fitting procedure, plot the results.

In [ ]:
# Verify the currently loaded data is still for ID 10013682
assert ID == 10013682

# Read in the best-fit redshift and its uncertainty
work_out_fit_values_and_f_esc_Lya_as_before()

# Show the best-fit high-resolution model curve
plot_spectra_and_best_fit_model_as_before()